# Multiple Linear Regression

## What we will accomplish

In this notebook we will:
- Introduce the multiple linear regression model,
- Show how to fit the model using the <i>normal equation</i>,
- Fit a model with `sklearn`.
- Show how to use the normal equations to make a custom class which mimics the sklearn model object.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import set_style

set_style("whitegrid")

## The multiple linear regression model

Suppose there is a quantitative variable you want to predict/model called $y$ and a set of $p$ features $x_1, x_2, \dots x_p$, then the multiple linear regression model regressing $y$ on $x_1, \dots, x_p$ is:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_m x_p + \epsilon = \vec{x} \cdot \vec{\beta} + \epsilon,
$$

where $\beta_0, \dots, \beta_p \in \mathbb{R}$ are constants, $\vec{\beta}$ is the $(p+1)$-vector of the $\beta_i$ in numerical order, 

$$
\vec{x} = \left(1, x_1, x_2, \dots, x_p \right)^\top,
$$

and $\epsilon \sim N(0,\sigma^2)$ is an error term independent of $\vec{x}$.  Note that we have "padded" $\vec{x}$ with an initial one to capture the constant term.

### Fitting the model

Suppose that we have $n$ observations $(\vec{x}_i, y_i)$.   We can package these into an $n \times (p+1)$ matrix $X$ and a $n$-vector $\vec{y}$

$$
X = \begin{bmatrix}
1 & x_{11} & x_{12} & ... & x_{1p}\\
1 & x_{21} & x_{22} & ... & x_{2p}\\
  &        &        & \vdots &    \\
1 & x_{n1} & x_{n2} & ... & x_{np}\\  
\end{bmatrix}

\hphantom{fdsfds}

\vec{y} = 
\begin{bmatrix}
y_1\\y_2\\ \vdots \\ y_n
\end{bmatrix}
$$

In order to fit a multiple linear regression model regressing $y$ on $\vec{x}$ using this data we return to the mean square error.

$$
\operatorname{MSE}(\vec{\beta}) = \frac{1}{n} \sum_{i=1}^n \left(y_i - f_{\vec{\beta}}(\vec{x}_i)\right)^2 = \frac{1}{n} \sum_{i=1}^n \left(y_i - \vec{x}_i^\top \vec{\beta}\right)^2,
$$

we can rewrite this using some linear algebra as:

$$
\operatorname{MSE}(\vec{\beta})  = \frac{1}{n} \left\vert \vec{y} - X \vec{\beta}\right\vert^2
$$

We want to find the value of the parameter $\vec{\beta}$ which minimizes the MSE.

As seen in Math Hour,  we can minimize it in two ways:

* Using multivariable differential calculus:  find gradient of the MSE with respect of $\beta$ and set it equal to zero.
* Using linear algebra:  view this geometrically as projecting $\vec{y}$ into the subspace spanned by the columns of $X$.

Here is a short derivation using the linear algebra approach:  We are looking for $\hat{\beta}$ so that $\vec{y} - X \hat{\beta}$ is orthogonal to $\operatorname{Im}(X)$.  So 

$$
\begin{align*}
& X^\top (\vec{y} - X \hat{\beta}) = 0\\
& X^\top \vec{y} - X^\top X \hat{\beta} = 0 \\
& X^\top X \hat{\beta} = X^\top y\\\
& \hat{\beta} = (X^\top X)^{-1} X^\top y
\end{align*}
$$

This is the <i>ordinary least squares</i> estimate of the coefficient vector $\hat{\beta}$. Note that this formula is also sometimes called the <i>normal equation</i>.

Note:  $X^\top X$ is invertible if the columns of $X$ are linearly independent.  When this is not true we say that $X$ suffers from "exact multicollinearity".  Even if the columns of $X$ are linearly independent we can still have issues if they are "close" (i.e. if the condition number of $X$ is large).

### A multiple linear regression model for abalone age

We will demonstrate how to fit this model using `sklearn` with the abalone data.

We will then write a naive custom class which functions similarly to the sklearn module to get a feel for what is going on "under the hood".

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
abalone = fetch_ucirepo(id=1) 
  
# data (as pandas dataframes) 
X = abalone.data.features 
y = abalone.data.targets 

continuous_features = ['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight']

X.loc[:,continuous_features] *= 200
y += 1.5

### Using `sklearn`

In [ ]:
## import the LinearRegression object


In [ ]:
## Make the model object

model = 

## Fit the model object
## note I do NOT have to use reshape here
## because X_train is a 2D np.array


In [ ]:
## look at coef and intercept


In [ ]:
## Make a prediction


### Implementing a custom linear regression class

To give you a feeling for what is going on "under the hood" of an sklearn model object, we will write a minimal custom linear regression model object.  The actual sklearn implementation is much more sophisticated.

In [ ]:
class CustomLinearRegression():
    '''A minimal custom linear regression model object'''
    def __init__(self):
        self.beta = None
    def fit(self,X,y):
        '''Finds self.beta using the normal equation.  
            X and y must be numpy arrays.
            X must have shape (num_samples, num_features)
            y must have shape (num_samples,)'''
        X = np.hstack([np.ones((X.shape[0],1)), X]) # adds column of ones to X
        XtX = np.dot(X.transpose(),X) 
        XtXinv = np.linalg.inv(XtX)
        Xty = np.dot(X.transpose(),y)
        self.beta = np.dot(XtXinv, Xty)
    def predict(self,X):
        X = np.hstack([np.ones((X.shape[0],1)), X]) # adds column of ones to X
        return np.dot(X, self.beta)

In [ ]:
custom_model = CustomLinearRegression()
custom_model.fit(X[continuous_features],y)

In [ ]:
custom_model.beta

In [ ]:
model.intercept_, model.coef_

#### Warning

In the simple linear regression notebook we noted many caveats.  Those remarks apply equally well here!  If our goal is inference we have a new danger:  **multicollinearity** of the predictors!  Intuitively if some of our predictors are themselves strongly correlated (say length and diameter) then it becomes difficult to disambiguate their effects, leading to wider confidence intervals for the coefficients of these parameters.  Mathematically, multicollinearity means that $X^\top X$ has a large condition number. As a result changes in $y$ can lead to large changes in $(X^\top X)^{-1}(X^\top y)$. We will return to these issues when we get to inference in lectures 5 and 6.

Another major warning:  we didn't do any Exploratory Data Analysis (EDA) on our other predictors.  Some of these other predictors could, conceivably, have non-linear relationships with the outcome in which case applying "vanilla" linear regression would be inappropriate.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.  Modified by Steven Gubkin 2024.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)